In [1]:
import pandas

titanic = pandas.read_csv("https://databootcamp.nyc3.digitaloceanspaces.com/titanic_2.csv")

## Tranformação de colunas do tipo **String**

Podemos tratar as Strings fazendo um _apply_... **mas existe uma maneira mais simples para aplicar algumas tranformações!**

Quando em uma coluna é tipo `object` podemos acessar as funções do tipo `string` usando a propriedade `str`.

In [14]:
titanic["Name"].str

**2.1 - `replace` em todos os dados de uma coluna**

Vamos começar com uma tranformação simples: retirar as vírgulas.

`Braund, Mr. Owen Harris => Braund Mr. Owen Harris`

In [15]:
titanic["Name"].str.replace(",","").head(10)

0                               Braund Mr. Owen Harris
1    Cumings Mrs. John Bradley (Florence Briggs Tha...
2                                Heikkinen Miss. Laina
3          Futrelle Mrs. Jacques Heath (Lily May Peel)
4                              Allen Mr. William Henry
5                                      Moran Mr. James
6                               McCarthy Mr. Timothy J
7                        Palsson Master. Gosta Leonard
8     Johnson Mrs. Oscar W (Elisabeth Vilhelmina Berg)
9                   Nasser Mrs. Nicholas (Adele Achem)
Name: Name, dtype: object

**2.1 - `split` em todos os dados de uma coluna**

Usando isso vamos conseguir separar o sobrenome.

Primeiro como funciona o split?

In [6]:
name = "Meu, Exemplo"
name.split(",")

['Meu', ' Exemplo']

O `split` divide as strings criando uma lista para cada elemento, a divisão para cada elemento é via um separador nesse caso `,` por padrão o divisor é o espaço em branco.

Ao aplicar o `split` na coluna o retorno é uma coluna com uma `lista` dentro.

In [7]:
titanic["Name"].str.split(",")

0                             [Braund,  Mr. Owen Harris]
1      [Cumings,  Mrs. John Bradley (Florence Briggs ...
2                              [Heikkinen,  Miss. Laina]
3        [Futrelle,  Mrs. Jacques Heath (Lily May Peel)]
4                            [Allen,  Mr. William Henry]
                             ...                        
886                             [Montvila,  Rev. Juozas]
887                      [Graham,  Miss. Margaret Edith]
888          [Johnston,  Miss. Catherine Helen "Carrie"]
889                             [Behr,  Mr. Karl Howell]
890                               [Dooley,  Mr. Patrick]
Name: Name, Length: 891, dtype: object

Nós podemos criar uma coluna para cada termo da lista usando o `expand`.

In [13]:
dataframe_nome_dividido = titanic["Name"].str.split(",", expand=True).head(20)
dataframe_nome_dividido

,0,1
0,Braund,Mr. Owen Harris
1,Cumings,Mrs. John Bradley (Florence Briggs Thayer)
2,Heikkinen,Miss. Laina
3,Futrelle,Mrs. Jacques Heath (Lily May Peel)
4,Allen,Mr. William Henry
5,Moran,Mr. James
6,McCarthy,Mr. Timothy J
7,Palsson,Master. Gosta Leonard
8,Johnson,Mrs. Oscar W (Elisabeth Vilhelmina Berg)
9,Nasser,Mrs. Nicholas (Adele Achem)


Com isso criamos duas colunas, uma com o sobrenome `0` e outra com o nome completo. 

Vamos associar a coluna **sobrenome** ao nosso dataframe.

In [10]:
titanic["Sobrenome"] = dataframe_nome_dividido[0]

Vamos ver como ficou.

In [11]:
nome_completo_sobrenome = ["Name", "Sobrenome"]

titanic[nome_completo_sobrenome].head()

,Name,Sobrenome
0,"Braund, Mr. Owen Harris",Braund
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Cumings
2,"Heikkinen, Miss. Laina",Heikkinen
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Futrelle
4,"Allen, Mr. William Henry",Allen


### **Desafio:** Vamos criar uma coluna com o nome da mulher `NomeMulher`.

**Infelizmente, nos dados do Titanic o nome do marido aparecia na frente do nome da mulher, o nome da mulher aparecia entre parênteses `()`**

![alt text](https://static.vix.com/pt/sites/default/files/styles/large/public/a/atrasada-relogio-pulso-1116-1400x800.jpg?itok=qv3gUH6U)

In [21]:
nomes = titanic["Name"].str.replace(")", "").str.split("(", expand=True)

,0,1,2
0,"Braund, Mr. Owen Harris",None,None
1,"Cumings, Mrs. John Bradley",Florence Briggs Thayer,None
2,"Heikkinen, Miss. Laina",None,None
3,"Futrelle, Mrs. Jacques Heath",Lily May Peel,None
4,"Allen, Mr. William Henry",None,None
...,...,...,...
886,"Montvila, Rev. Juozas",None,None
887,"Graham, Miss. Margaret Edith",None,None
888,"Johnston, Miss. Catherine Helen ""Carrie""",None,None
889,"Behr, Mr. Karl Howell",None,None


## Tabela de frequência

![pivot](http://web.cse.ohio-state.edu/~reeves.92/CSE4194/Lab6Pivot_files/image041.png)

Para entender a correlação entre duas variáveis precisamos colocá-las lado a lado.

O Pandas nos oferece uma funcionalidade de crosstab (tabela de frequência).


**Vamos fazer a tabela de frequência entre os sobrenomes e as cabine, para validarmos se cada cabine possuia apenas uma família**

In [22]:
pandas.crosstab(titanic["Cabin"], titanic["Sobrenome"]).head(10)

Sobrenome,Bonnell,Cumings,Futrelle,McCarthy,Sandstrom
Cabin,,,,,
C103,1,0,0,0,0
C123,0,0,1,0,0
C85,0,1,0,0,0
E46,0,0,0,1,0
G6,0,0,0,0,1


Nós conseguimos fazer operações em uma `crosstab`, como por exemplo somar a quantidade de famílias (sobrenomes diferentes) em uma cabine.

In [23]:
tabela_frequencia_cabine_sobrenome = pandas.crosstab(titanic["Cabin"], titanic["Sobrenome"])

(tabela_frequencia_cabine_sobrenome > 0).sum(axis=1)

Cabin
C103    1
C123    1
C85     1
E46     1
G6      1
dtype: int64

Usamos o `axis=1` para somar as famílias por cabine.

Podemos ordenar para ver as cabines com mais famílias.

In [24]:
familias_diferentes_por_cabine = (tabela_frequencia_cabine_sobrenome > 0).sum(axis=1)
familias_diferentes_por_cabine.sort_values(ascending=False).head()

Cabin
G6      1
E46     1
C85     1
C123    1
C103    1
dtype: int64

Se quisermos inverter e usar como eixo as famílias basta usar o `axis=0`

In [25]:
cabines_diferentes_por_familia = (tabela_frequencia_cabine_sobrenome > 0).sum(axis=0)
cabines_diferentes_por_familia.sort_values(ascending=False).head()

Sobrenome
Sandstrom    1
McCarthy     1
Futrelle     1
Cumings      1
Bonnell      1
dtype: int64